<a href="https://colab.research.google.com/github/ChibuzoKelechi/GoogleColab_notebooks/blob/main/imgclassifybeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kaggle

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [3]:
!kaggle datasets download -d cashutosh/gender-classification-dataset

 97% 260M/269M [00:02<00:00, 96.8MB/s]
100% 269M/269M [00:02<00:00, 99.3MB/s]


In [ ]:
import zipfile

# Define the path to your zip file
file_path = '/content/gender-classification-dataset.zip'  # Replace 'your_file.zip' with your file's name

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kaggle')  # Replace 'destination_folder' with your desired folder

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
#import cv2
import PIL
import pathlib

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras as tfkeras
from tensorflow.keras import datasets, layers, models, callbacks
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator

In [ ]:
# Variables
data_path = ''
data_dir = pathlib.Path(data_path)

width=200
height=200

gender_labels_dict = {
    'male': 0,
    'female': 1
}

x = []
y = []

print(data_dir)

In [ ]:
# Data
# for gender, images in gender_image_dict.items():
#     print(gender)
#     for image in images:
#         img = cv2.imread(str(image))
#         if isinstance(img,type(None)):
#             continue

#         elif ((img.shape[0] >= height) and  (img.shape[1] >=width)):
#             resized_img = cv2.resize(img,(width,height))
#             x.append(resized_img)
#             y.append(gender_labels_dict[gender])
#         else:
#             continue
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size
)


In [ ]:
# Convert to array, split data
# x = np.array(x)
# y = np.array(y)

# print(x.shape)
# print(y.shape)

# x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.1)

# print(len(x_train),len(y_train))
# print(len(x_test),len(y_test))

# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)

In [ ]:
# Model
channels = 3

image_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    layers.Conv2D(64, (3,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout(0.2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

image_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# image_model.summary()

In [ ]:
# Training
earlystop = tfkeras.callbacks.EarlyStopping(
    patience=5,
    verbose=1
)

history = image_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[earlystop]
)

In [ ]:
test_acc = image_model.evaluate(val_ds)

print(f'Loss: {test_acc[0]}')
print(f'Accuracy: {100*(test_acc[1])}%')

In [ ]:
# Loss vs Accuracy plot

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# image_model.save('gender_image_classifier.h5')
import os
image_model.save(os.path.join('models','gender_image_classifier1.h5'))

# os.path

In [ ]:


classes = ["male","female"]
img_path = "/kaggle/input/testing-images/julie.jpg"
fileImage = Image.open(img_path).convert("RGB").resize([width,height],Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width,height,3)

# prepare pixel data
myimage = myimage.astype('float32')
myimage = myimage/255.

plt.figure(figsize = (4,2))
plt.imshow(image)





In [ ]:
pred_arr = image_model.predict(myimage)

pred_class = pred_arr.argmax()
probability = pred_arr.max()

print(pred_class)
print(probability)